# 🍽 Food Delivery Data Integration & Analysis

## 📌 Project Description
This notebook demonstrates a real-world data engineering and analytics workflow by
combining data from multiple formats (CSV, JSON, SQL) into a single analytical dataset.

The final dataset is used to analyze:
- Order trends
- User behavior
- City-wise and cuisine-wise performance
- Membership impact on revenue

📁 **Final Output:** `final_food_delivery_dataset.csv`  
📌 This dataset acts as the **single source of truth** for all analysis.


## 1️⃣ Import Required Libraries

We use:
- **pandas** for data manipulation
- **sqlite3** to read restaurant data from SQL database


In [2]:
import pandas as pd
import sqlite3


## 2️⃣ Load Orders Data (CSV)

This dataset contains **transactional order-level data**.
Each row represents a single order placed on the platform.

### Key Fields:
- order_id
- user_id
- restaurant_id
- order_date
- order_amount


In [3]:
orders_df = pd.read_csv("C:/Users/bshas/OneDrive/Desktop/data/orders.csv")


## 3️⃣ Load Users Data (JSON)

This dataset contains **user master information**.
Each user can place multiple orders.

### Key Fields:
- user_id
- user_name
- city
- membership_type (Gold / Regular)


In [4]:
users_df = pd.read_json("C:/Users/bshas/OneDrive/Desktop/data/users.json")


## 4️⃣ Create Restaurants Database from SQL Script

This step executes the SQL file to create the restaurants table
inside the SQLite database.


In [5]:
conn = sqlite3.connect("C:/Users/bshas/OneDrive/Desktop/data/restaurants.db")
cursor = conn.cursor()

# Check if table already exists
cursor.execute("""
SELECT name FROM sqlite_master 
WHERE type='table' AND name='restaurants';
""")

if cursor.fetchone() is None:
    with open("C:/Users/bshas/OneDrive/Desktop/data/restaurants.sql", "r") as file:
        sql_script = file.read()
    cursor.executescript(sql_script)
    conn.commit()
    print("✅ restaurants table created successfully")
else:
    print("ℹ️ restaurants table already exists, skipping creation")

conn.close()


ℹ️ restaurants table already exists, skipping creation


### 🔍 Verify Tables in Database

This step confirms that the restaurants table
was successfully created.


In [6]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("C:/Users/bshas/OneDrive/Desktop/data/restaurants.db")

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

print(tables)
conn.close()


          name
0  restaurants


## Load Restaurants Data from SQLite Database

The restaurant master data is stored in a SQLite database to simulate
a real-world production environment.

In this step:
- We establish a connection to the SQLite database
- Read the `restaurants` table into a Pandas DataFrame
- Close the database connection after loading the data

This dataset provides key restaurant-level attributes such as:
- Restaurant name
- City
- Cuisine type
- Rating


In [7]:
conn = sqlite3.connect(
    "C:/Users/bshas/OneDrive/Desktop/data/restaurants.db"
)

restaurants_df = pd.read_sql(
    "SELECT * FROM restaurants",
    conn
)

conn.close()

restaurants_df.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## 5️⃣ Merge Orders with Users Data

### Join Logic:
- orders.user_id → users.user_id
- Join Type: **LEFT JOIN**

🔹 Why LEFT JOIN?  
To ensure **all orders are retained**, even if user details are missing.


In [8]:
orders_users_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)


## 6️⃣ Merge with Restaurants Data

### Join Logic:
- orders.restaurant_id → restaurants.restaurant_id
- Join Type: **LEFT JOIN**

This step enriches each order with restaurant details
such as cuisine, city, and rating.


In [9]:
final_df = pd.merge(
    orders_users_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## 8️⃣ Export Final Integrated Dataset

After successfully merging order, user, and restaurant data,
the final integrated dataset is exported as a CSV file.

📁 **Output File:** `final_food_delivery_dataset.csv`

This file serves as the **single source of truth** for:
- Exploratory Data Analysis (EDA)
- Business insights
- Dashboards and visualizations
- Hackathon and interview questions

The dataset can now be reused without re-running the entire data pipeline.


In [12]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)
final_df = pd.read_csv("final_food_delivery_dataset.csv")


## 📊 Business Questions & Analytical Insights

In this section, we answer a set of business-driven analytical questions
using the final integrated dataset (`final_food_delivery_dataset.csv`).

Each question focuses on extracting actionable insights related to:
- Revenue performance
- User behavior
- Membership impact
- Cuisine and restaurant effectiveness
- Time-based trends

All answers are derived **exclusively from the final dataset**, which serves
as the single source of truth for this analysis.


In [16]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

### ❓ Question 2


In [19]:
(
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [20]:
high_spenders = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

(high_spenders > 1000).sum()


np.int64(2544)

In [22]:
(
    final_df
    .groupby("rating_range", observed=False)["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [23]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [24]:
final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_name_y", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("restaurant_count")


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [25]:
round(
    (final_df[final_df["membership"] == "Gold"].shape[0]
     / final_df.shape[0]) * 100
)


50

In [29]:
(
    final_df
    .groupby("restaurant_name_y")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("total_amount", "count")
    )
    .query("total_orders < 20")
    .sort_values("avg_order_value", ascending=False)
)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [31]:
(
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [36]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)

final_df.assign(
    quarter=final_df["order_date"].dt.quarter
).groupby("quarter")["total_amount"].sum().idxmax()


np.int32(3)

In [37]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [38]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [39]:
final_df["user_id"].nunique()


2883

In [40]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [41]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [42]:
# Step 1: Find top revenue city among Gold members
top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# Step 2: Count orders in that city by Gold members
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]


1337